In [46]:
# 读取excel文件，输出表头
import pandas as pd

datas = pd.read_excel('InformationClassification_Report_2.xlsx')

table_ICL = datas.columns[datas.columns.tolist().index('ICL ID'):datas.columns.tolist().index('IOB ID')]
table_IOB = datas.columns[datas.columns.tolist().index('IOB ID'):]
table_IOB_FOREIGN = datas.columns[datas.columns.tolist().index('ICL ID'):datas.columns.tolist().index('ICL ID')+1]


print(f"table_ICL: {table_ICL}")
print(f"table_IOB: {table_IOB}")
print(f"table_IOB_FOREIGN: {table_IOB_FOREIGN}")

# 读取每一行数据，保存为元组
icl_data = []
iob_data = []
id = -1
for index, row in datas.iterrows():
    # 判断row[table_ICL]是否包含nan元素，如果有，对应的元素变为“空值”
    # 处理ICL数据
    icl_row = row[table_ICL].tolist()

    icl_row = ['空值' if pd.isna(x) else x for x in icl_row]
    # 判断icl_row是否全部为nan，如果是，跳过
    # icl_row.insert(0, id)
    icl_data.append(icl_row)

    # 处理IOB数据
    iob_row = row[table_IOB].tolist()
    iob_row = ['空值' if pd.isna(x) else x for x in iob_row]
    # iob_row.insert(0, id)
    iob_data.append(iob_row)
    
    # 添加外键数据
    iob_data[-1].append(row[table_IOB_FOREIGN].tolist()[0])

    id += 1

# icl_data去重
icl_data = list(map(list, set(map(tuple, icl_data[1:]))))
# print(f"icl_data: {icl_data}")
# print(f"len_data_icl: {len(icl_data)}")


# 在icl和iob数据中添加id
for i in range(len(icl_data)):
    icl_data[i].insert(0, i)

for i in range(len(iob_data)):
    iob_data[i].insert(0, i-1)

# 输出处理后的数据
print("\nICL数据示例:")
print(icl_data[0])  # 打印前1条ICL数据
print(f"len_data_icl[0]: {len(icl_data[1:2][0])}")

print("\nIOB数据示例:")
print(iob_data[1:2])  # 打印前1条IOB数据
print(f"len_data_iob[0]: {len(iob_data[1:2][0])}")

print(f"\nlen_data_icl: {len(icl_data)}")
print(f"len_data_iob: {len(iob_data)}")
# print(f"icl_data: {icl_data}")
# print(f"iob_data: {iob_data}")

# # 遍历输出icl_data的第一个数据
# for i in range(len(icl_data)):
#     print(f"icl_data[{i}]: {icl_data[i][0]}")




table_ICL: Index(['ICL ID', 'ICL Name', 'Description', 'Category', 'Information Owner',
       'Record Creator', 'Status ', 'ICL Security Class',
       'ICL Classification Result\n (Confidentiality)',
       'ICL Classification Result \n(Integrity)',
       'ICL Classification Result\n (Availability)', 'Submit Date',
       'Approval Date'],
      dtype='object')
table_IOB: Index(['IOB ID', 'IOB Name', 'Level 1', 'Level 2', 'Level 3',
       'Method of Classification', 'IOB Security Class',
       'IOB Classification Result\n (Confidentiality)',
       'IOB Classification Result\n (Integrity)',
       'IOB Classification Result\n (Availability)'],
      dtype='object')
table_IOB_FOREIGN: Index(['ICL ID'], dtype='object')

ICL数据示例:
[0, 'ICL-0002401', 'Change management_M-gAMS', 'It defines the business flow of change management for buyrer&KIFA QMT in BBT-M', 'Business Process', 'Chen Bo, BBT-M-2', 'Tang Guoming, BBT-M-2', 'Approved', 'Special Protection (SP)', 'Strictly Confidential', 

In [49]:
import sqlite3

# 连接到SQLite数据库（如果数据库不存在，则会创建一个新的数据库）
conn = sqlite3.connect('InformationClassification_Report.db')
cursor = conn.cursor()

# 创建 table_ICL 表
cursor.execute('''
CREATE TABLE IF NOT EXISTS table_ICL (
    id INTEGER PRIMARY KEY AUTOINCREMENT, -- ID primary key
    ICL_ID TEXT, -- ICL ID
    ICL_Name TEXT NOT NULL, -- ICL 信息名称
    Description TEXT, -- 信息描述
    Category TEXT, -- 信息所属类别
    Information_Owner TEXT, -- 信息所有者
    Record_Creator TEXT, -- 信息记录创建者
    Status TEXT, -- 信息状态，可选值包括Approved和Rejected
    ICL_Security_Class TEXT, -- ICL 信息分类安全等级，可选值包括Special Protection (SP)和Basic Protection (BP)
    ICL_Classification_Result_Confidentiality TEXT, -- ICL 分类结果（机密性），可选值包括Strictly Confidential、Confidential、Non Public
    ICL_Classification_Result_Integrity TEXT, -- ICL 分类结果（完整性），可选值包括High Integrity、Medium Integrity、Low Integrity
    ICL_Classification_Result_Availability TEXT, -- ICL 分类结果（可用性），可选值包括High Availability、Medium Availability、Low Availability
    Submit_Date TEXT, -- 提交日期
    Approval_Date TEXT -- 批准日期
)
''')

# 创建 table_IOB 表，并定义与 table_ICL 的外键关系
cursor.execute('''
CREATE TABLE IF NOT EXISTS table_IOB (
    id INTEGER PRIMARY KEY AUTOINCREMENT, -- ID primary key
    IOB_ID TEXT, -- IOB ID
    IOB_Name TEXT NOT NULL, -- IOB 名称
    Level_1 TEXT, -- IOB 一级分类
    Level_2 TEXT, -- IOB 二级分类
    Level_3 TEXT, -- IOB 三级分类
    Method_of_Classification TEXT, -- IOB 分类方法，包括General Questionnaire和Personal Information Questionnaire
    IOB_Security_Class TEXT, -- IOB 分类安全等级，包括Basic Protection (BP)和Special Protection (SP)
    IOB_Classification_Result_Confidentiality TEXT, -- IOB 分类结果（机密性），可选值包括Strictly Confidential、Confidential、Non Public
    IOB_Classification_Result_Integrity TEXT, -- IOB 分类结果（完整性），可选值包括High Integrity、Medium Integrity、Low Integrity
    IOB_Classification_Result_Availability TEXT, -- IOB 分类结果（可用性），可选值包括High Availability、Medium Availability、Low Availability
    ICL_ID TEXT, -- ICL ID（外键）
    FOREIGN KEY (ICL_ID) REFERENCES table_ICL(ICL_ID) -- 外键约束，关联 ICL ID
)
''')

# 提交事务
conn.commit()


In [50]:
print(icl_data[0])
print(iob_data[1])

[0, 'ICL-0002401', 'Change management_M-gAMS', 'It defines the business flow of change management for buyrer&KIFA QMT in BBT-M', 'Business Process', 'Chen Bo, BBT-M-2', 'Tang Guoming, BBT-M-2', 'Approved', 'Special Protection (SP)', 'Strictly Confidential', 'Medium Integrity', 'Medium Availability', '09/06/2023 04:25 PM', '09/07/2023 03:32 PM']
[0, 'IOB-0002435', 'Process document signed version', 'Purchasing&Supplier Data', 'Purchasing & Part Quality', 'Purchasing/ Part Quality', 'General Questionnaire', 'Basic Protection (BP)', 'Confidential', 'Medium Integrity', 'Medium Availability', 'ICL-0001114']


In [51]:

# 插入到 table_ICL,已去重
cursor.executemany('''
INSERT INTO table_ICL (id, ICL_ID, ICL_Name, Description, Category, Information_Owner, Record_Creator, Status, ICL_Security_Class, ICL_Classification_Result_Confidentiality, ICL_Classification_Result_Integrity, ICL_Classification_Result_Availability, Submit_Date, Approval_Date)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', icl_data)


# 插入到 table_IOB 的数据样例
cursor.executemany('''
INSERT INTO table_IOB (id, IOB_ID, IOB_Name, Level_1, Level_2, Level_3, Method_of_Classification, IOB_Security_Class, IOB_Classification_Result_Confidentiality, IOB_Classification_Result_Integrity, IOB_Classification_Result_Availability, ICL_ID)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
''', iob_data[1:])

# 提交事务
conn.commit()

In [44]:
# # 查询 table_ICL有多少条数据
# cursor.execute('SELECT COUNT(*) FROM table_ICL')
# print(cursor.fetchall())

# # 查询 table_IOB有多少条数据
# cursor.execute('SELECT COUNT(*) FROM table_IOB')
# print(cursor.fetchall())



import sqlite3

# 连接到SQLite数据库（如果数据库不存在，则会创建一个新的数据库）
conn = sqlite3.connect('InformationClassification_Report.db')
cursor = conn.cursor()

# 查询 table_ICL有多少条数据
cursor.execute("""
               SELECT
*
FROM
  table_ICL
WHERE
  Description = 'Escalation Procedure'
LIMIT
  50         """)
print(cursor.fetchall())


# 关闭连接
conn.close()


[]
